## Imports

In [121]:
import os
import pandas as pd
import numpy as np
import requests

In [149]:
import winreg
import shutil

In [182]:
from datetime import date, timedelta

In [123]:
from selenium import webdriver
from bs4 import BeautifulSoup

## Setup

In [124]:
convert_dict = {
    '01': 'jan',
    '02': 'fev',
    '03': 'mar',
    '04': 'abr',
    '05': 'mai',
    '06': 'jun',
    '07': 'jul',
    '08': 'ago',
    '09': 'set',
    '10': 'out',
    '11': 'nov',
    '12': 'dez'
}

def converte_data(ds):
    #given a data string, converts the string to the file format
    month = convert_dict[ds[2:4]]
    return 'd'+ds[-2:] + month + ds[:2] + '.xls'

In [135]:
url = 'https://www.anbima.com.br/informacoes/merc-sec-debentures/default.asp'
driver = webdriver.Chrome()
driver.get(url)

In [136]:
datainput = driver.find_element_by_xpath('//*[@id="cinza50"]/form/div/fieldset/table/tbody/tr/td/input[2]')
datainput.clear()
data = '17122019'
datainput.send_keys(data)


In [137]:
consultbut = driver.find_element_by_xpath('//*[@id="cinza50"]/form/div/table/tbody/tr/td/img')
consultbut.click()

In [138]:
downbut = driver.find_element_by_xpath('/html/body/table/tbody/tr/td/div/table[2]/tbody/tr/td/table/tbody/tr/td/a[1]')
downbut.click()

In [130]:
driver.execute_script('window.history.go(-1)')

## Pegando as datas

In [162]:
today = date.today()

In [163]:
days = [today - timedelta(days = j+1) for j in range(5)] 

In [164]:
days = [d.strftime('%d/%m/%Y').replace('/','') for d in days]

In [165]:
days

['19122019', '18122019', '17122019', '16122019', '15122019']

## Lidando com os Downloads

In [118]:
down_days = list(map(converte_data, days))

In [119]:
down_days

['d19dez20.xls',
 'd19dez19.xls',
 'd19dez18.xls',
 'd19dez17.xls',
 'd19dez16.xls']

## Tentando acessar servs

In [152]:
file = 'd19dez17.xls'

In [153]:
def get_download_path():
    sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
    downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
    with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            location = winreg.QueryValueEx(key, downloads_guid)[0]
    return location

In [154]:
down = get_download_path()

In [155]:
file_path = os.path.join(down, file)

In [158]:
server_path = 'Z:\Structuring\Credit Analysis\Debentures\Transfer'

In [159]:
server_dest_path = os.path.join(server_path, file)

In [161]:
filemove = shutil.copyfile(file_path, server_dest_path)

#### lidando com os dias uteis

In [192]:
holid_file = 'holidays.xlsx'
holid = pd.read_excel(os.path.join(os.getcwd(),holid_file))

In [193]:
holid = list(holid['Data'])

In [197]:
holid = [date(h.year,h.month,h.day).strftime('%d/%m/%Y').replace('/','') for h in holid]

In [206]:
dys = []
j = 0
while len(dys)<5:
    dy = today - timedelta(days = j + 1)
    if dy not in holid:
        dys.append(dy)
    j = j + 1

In [207]:
dys

[datetime.date(2019, 12, 19),
 datetime.date(2019, 12, 18),
 datetime.date(2019, 12, 17),
 datetime.date(2019, 12, 16),
 datetime.date(2019, 12, 15)]